In [276]:
"""
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
"""

'\n%tensorflow_version 2.x\nimport tensorflow as tf\nprint("Tensorflow version " + tf.__version__)\n\ntry:\n  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection\n  tf.config.experimental_connect_to_cluster(tpu)\n  tf.tpu.experimental.initialize_tpu_system(tpu)\n  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)\n  print(\'Running on TPU \', tpu.cluster_spec().as_dict()[\'worker\'])\nexcept ValueError:\n  raise BaseException(\'ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!\')\n'

In [277]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
name   ="M15_2008-2022_DataFrame"
#df=pd.read_csv("EURUSD.csv")
df = pd.read_csv('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'.csv', engine='python',index_col=[0])
    
#print("Number of rows and columns:", df.shape)
#df = df.drop('Adj Close',axis=1)
#df = df.drop('Volume',axis=1)
df.tail(10)

,Date,Open,High,Low,Close
346478,2021-12-31 20:45:00,1.13811,1.13847,1.13806,1.13842
346479,2021-12-31 21:00:00,1.13842,1.13853,1.13830,1.13842
346480,2021-12-31 21:15:00,1.13841,1.13850,1.13818,1.13821
346481,2021-12-31 21:30:00,1.13820,1.13847,1.13809,1.13810
346482,2021-12-31 21:45:00,1.13810,1.13836,1.13793,1.13822
346483,2021-12-31 22:00:00,1.13822,1.13838,1.13815,1.13836
346484,2021-12-31 22:15:00,1.13836,1.13844,1.13825,1.13828
346485,2021-12-31 22:30:00,1.13828,1.13847,1.13802,1.13844
346486,2021-12-31 22:45:00,1.13844,1.13845,1.13752,1.13755
346487,2021-12-31 23:00:00,1.13755,1.13755,1.13755,1.13755


In [278]:
#sc_x = MinMaxScaler(feature_range = (0, 1))
#sc_y = MinMaxScaler(feature_range = (1, 100000))

df_y = df.shift(-1, axis = 0)
df_y =df_y.dropna()
df_y = df_y.drop('Date',axis=1)
#df_y = df_y*1000
#df_y = sc_x.fit_transform(df_y)

df_x = df.shift(1,axis=0)
df_x = df_x.dropna()
df_x = df_x.drop('Date',axis=1)
#df_x = df_x*1000
#df_x = sc_x.fit_transform(df_x)
df=0
df_x = pd.DataFrame(df_x)
df_y = pd.DataFrame(df_y)
df_x,df_y

#a = sc_x.inverse_transform(df_x)
#b = sc_y.inverse_transform(df_y)


(           Open     High      Low    Close
 1       1.47220  1.47230  1.47170  1.47190
 2       1.47200  1.47290  1.47180  1.47290
 3       1.47280  1.47360  1.47280  1.47360
 4       1.47350  1.47380  1.47340  1.47340
 5       1.47350  1.47360  1.47320  1.47330
 ...         ...      ...      ...      ...
 346483  1.13810  1.13836  1.13793  1.13822
 346484  1.13822  1.13838  1.13815  1.13836
 346485  1.13836  1.13844  1.13825  1.13828
 346486  1.13828  1.13847  1.13802  1.13844
 346487  1.13844  1.13845  1.13752  1.13755
 
 [346487 rows x 4 columns],            Open     High      Low    Close
 0       1.47200  1.47290  1.47180  1.47290
 1       1.47280  1.47360  1.47280  1.47360
 2       1.47350  1.47380  1.47340  1.47340
 3       1.47350  1.47360  1.47320  1.47330
 4       1.47320  1.47360  1.47320  1.47360
 ...         ...      ...      ...      ...
 346482  1.13822  1.13838  1.13815  1.13836
 346483  1.13836  1.13844  1.13825  1.13828
 346484  1.13828  1.13847  1.13802  1.13844
 34

In [279]:
from sklearn.model_selection import train_test_split
train_size = ((len(df_x)*90)//100)

X_train, X_test, y_train, y_test = train_test_split(df_x.iloc[:train_size, 0:], df_y.iloc[:train_size, 0:], test_size=0.33,shuffle=False)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33,shuffle=False)
X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape 


((139983, 4), (102907, 4), (68948, 4), (139983, 4), (102907, 4), (68948, 4))

In [280]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 250))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 4))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])
model.summary()
# Fitting the RNN to the Training set

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_55 (LSTM)              (None, 4, 250)            252000    
                                                                 
 dropout_54 (Dropout)        (None, 4, 250)            0         
                                                                 
 lstm_56 (LSTM)              (None, 4, 250)            501000    
                                                                 
 dropout_55 (Dropout)        (None, 4, 250)            0         
                                                                 
 lstm_57 (LSTM)              (None, 4, 250)            501000    
                                                                 
 dropout_56 (Dropout)        (None, 4, 250)            0         
                                                                 
 lstm_58 (LSTM)              (None, 250)             

In [ ]:
model.fit(X_train, y_train, epochs = 100, batch_size = 4096,validation_data=(X_valid,y_valid))

Epoch 1/100
35/35 [==============================] - 20s 347ms/step - loss: 0.4037 - accuracy: 0.1921 - val_loss: 0.0067 - val_accuracy: 0.0476
Epoch 2/100
35/35 [==============================] - 10s 290ms/step - loss: 0.0148 - accuracy: 0.2502 - val_loss: 0.0130 - val_accuracy: 0.0000e+00
Epoch 3/100
35/35 [==============================] - 10s 292ms/step - loss: 0.0111 - accuracy: 0.2484 - val_loss: 0.0097 - val_accuracy: 0.0476
Epoch 4/100
35/35 [==============================] - 10s 292ms/step - loss: 0.0105 - accuracy: 0.2493 - val_loss: 0.0089 - val_accuracy: 0.0476
Epoch 5/100
35/35 [==============================] - 10s 293ms/step - loss: 0.0099 - accuracy: 0.2491 - val_loss: 0.0081 - val_accuracy: 0.9524
Epoch 6/100
35/35 [==============================] - 10s 293ms/step - loss: 0.0095 - accuracy: 0.2508 - val_loss: 0.0073 - val_accuracy: 0.0476
Epoch 7/100
35/35 [==============================] - 10s 293ms/step - loss: 0.0091 - accuracy: 0.2504 - val_loss: 0.0065 - val_accur

In [ ]:
#model.save('/content/drive/MyDrive/EURUSD_M1_CalisanModel/')

In [ ]:
predicted_stock_price = model.predict(X_test[:5])
#predicted_stock_price = sc_y.inverse_transform(predicted_stock_price)
#y_test = sc_y.inverse_transform(y_test)


In [ ]:
predicted_stock_price,y_test,df


array([[-0.00125543, -0.0012467 , -0.001641  , -0.00160765],
       [-0.00149559, -0.00109674, -0.00152141, -0.00140782],
       [-0.00104591, -0.00057671, -0.00121197, -0.00089814],
       [-0.00099577, -0.00054668, -0.00094182, -0.00090811],
       [-0.00105581, -0.00092672, -0.00144163, -0.00116804]])

In [ ]:
df.loc[:3118,'Date'].shape

In [ ]:
# Visualising the results
y_test2 = np.reshape(y_test,(len(y_test),4))
plt.plot(df.loc[:50, "Date"],df.loc[:50, "Open"], color = "red", label = "Real Price")
plt.plot(df.loc[:50, "Date"],predicted_stock_price[:51], color = "blue", label = "Predicted Price")
#plt.xticks(np.arange(0,len(df),50))
#plt.title('Price Prediction')
#plt.xlabel('Time')
#plt.ylabel('Price')
#plt.legend()
plt.show()

In [ ]:
"""arr = np.empty(shape=(1,1,1,1))
arr2 =  np.empty(shape=(1,1,1,1))
arr = [1.09925,1.09962,1.09871,1.09875]
#arr2 = [1.10096,1.10223,1.10017,1.10209]
arr = np.reshape(arr,(1,4))
#arr2 = np.reshape(arr2,(1,4))

a = model.predict(arr)
a
"""
#             open       high        low      close
#tahmin =  1.0896426,   1.0900719,   1.0892347,   1.0896202
#gerçek =  1.09049,     1.09174,     1.09010,     1.09170
#fark   = -0.00084736, -0.00166808, -0.00086529, -0.00207977